## Test of accelerated Monte Carlo ##

In [1]:
import sys
sys.path.insert(0, "../python/")

from particle import particle
from physics import em_physics
from cylinder import cylinder

import matplotlib.pyplot as plt
import numpy as np
from numba import jit
import pandas as pd

import json


### Define the Geometry ###

In [2]:
# define the geometry
radius = 50 # cm
height = 100 #cm
cryostat = cylinder(R=radius,h=height)

#radius = 40 # cm
#height = 80 #cm
radius = 49 #cm
height = 99 #cm
fiducial = cylinder(R=radius,h=height)

cylinder::__init__ Define cylinder with R= 50  and height= 100
cylinder::__init__ Define cylinder with R= 49  and height= 99


### Setup the photon physics ###

In [3]:
# define the physics
em = em_physics()

### Event generation ###

In [11]:
def generate_events(nevent,egamma,vrt,demax,nmax):
    #
    # output filename
    #
    
    np.random.seed(123457)

    
    #write_mode = 'json'
    write_mode = 'json'

    #mcout = '../mcdata/mc_std_noecut_1M' + '.' + write_mode 
    if vrt == None:
        mcout = '../mcdata/mc_'+str(vrt)+'_de'+str(demax)+'_N'+str(nevent) + '.' + write_mode 
    else:
        mcout = '../mcdata/mc_'+str(vrt)+'_de'+str(demax)+'_N'+str(nevent) + '_nmax'+str(nmax)+'.' + write_mode 


    #
    # open output data file
    #
    f = open(mcout,'w')
    if write_mode == 'json':
        f.write('[\n')    
    
    #
    # event generation loop
    #
    for ieve in range(nevent):

        if ieve%25000 == 0:
            print("generated ",ieve," events")
        #
        # make a particle
        #
        p = particle(type='gamma',
                     energy=egamma, 
                     geometry=cryostat, 
                     fiducial=fiducial, 
                     # vrt=None,
                     vrt=vrt,
                     edep_max=demax,
                     nscatter_max=nmax,
                     physics=em,
                     debug=False)

        #
        # propagate the particle and retrieve the intersection points
        #
        p.propagate()

        #
        # store data
        #
        if write_mode == 'txt':
            wstring = "{:d} {:d} {:7.6e} {:7.6f} {:7.6f} {:7.6f} {:7.6f} \n".format(ieve,len(p.xint), 
                                                                                    p.weight,p.edep,
                                                                                    p.x0start[0],p.x0start[1],p.x0start[2])
            f.write(wstring)

        for i in range(len(p.xint)):
            #
            # data structure for each energy deposit
            #
            if write_mode == 'json':
                dd = {'x':p.xint[i][0][0],'y':p.xint[i][0][1],'z':p.xint[i][0][2],'de':p.xint[i][1],
                      'w':p.weight,'n':p.nscatter}
                json.dump(dd,f, sort_keys=True, indent=8)
                f.write(',\n')
            else:
                wstring = "{:7.6f} {:7.6f} {:7.6f} {:7.6f} \n".format(p.xint[i][0][0],
                                                                      p.xint[i][0][1],
                                                                      p.xint[i][0][2],
                                                                      p.xint[i][1])
                f.write(wstring)

    #
    # close output data file
    #
    if write_mode == 'json':
        f.write('{}]')
    else:
        f.write('-1\n')

    f.close()

In [12]:
# !snakeviz my.profile

In [15]:
generate_events(10,2447.86, 'fiducial_scatter',2500.,2)

generated  0  events
random =  0.8414104718886751
random =  0.6552324999562392
random =  0.8162186300691359
random =  0.9885681151823003
random =  0.6548383143920027
random =  0.09164591828539925
random =  0.8304492668021675


In [20]:
generate_events(10,2447.86, None, 2500.,1)

generated  0  events
random =  0.9960838119932811
random =  0.9678457592556462
